In [ ]:
versioninfo()

In [ ]:
g=6.0
ENV["JULIA_CUDA_MEMORY_LIMIT"] = convert(Int, round(g * 1024 * 1024 * 1024))

In [ ]:
ENV["JULIA_DEBUG"]="all"

# New version of main

- [X] refactor the code of main1
- [X] use seeding
- [X] pre-gen graphs and splitting
- [X] use cloud GPU for training
- [X] I don't need correlation experiments anymore

In [ ]:
1+2

In [ ]:
include("../src/main.jl")

In [ ]:
main_data()

In [ ]:
main_ngraph()

In [ ]:
main_ersf124()

In [ ]:
main_ch3()

In [ ]:
function main_cnn()
    for d in [10,20,50],
        (prefix, model_fn,nsteps) in [
#             ("CNN", ()->cnn_model(2), 3e4),
            ("CNN2-$(now())", ()->cnn_model(2, 128, (5,5), (2,2)), 3e4),
#             ("CNN2", ()->cnn_model(2, 128, (3,3), (1,1)), 3e4)
        ]
        
        specs = []
        for gtype in [:ER, :SF],
            k in [1]
            push!(specs, DataSpec(d=d, k=k, gtype=gtype,
                    noise=:Gaussian, mat=:CH3,
                    ng=10000, N=1))
        end
        specs = Array{DataSpec}(specs)
        
        # print more frequently for CNN and FC to get more data to print
        test_throttle = if prefix == "EQ2" 10 else 1 end

        @info "training .." prefix d
        expID = exp_train(specs, model_fn,
                          # TODO I'll need to increase the training steps here
                          # CAUTION feed in the gtype in the model prefix
                          prefix="$prefix-ERSF-k1-d=$d", train_steps=nsteps,
                          test_throttle = test_throttle,
                          merge=true)
    end
end

In [ ]:
main_cnn()

In [ ]:
main_mat()

In [ ]:
main_ensemble_d()

# DEBUG

In [ ]:
main()

Ensemble

- [ ] I actually probably want to merge the datasets with the same d.
  This is because, I can merge the data to be more evenly distributed to fit the model.
- [ ] I probably want to adjust batch size to speed it up

In [ ]:
model = eq_model_fn() |> gpu

In [ ]:
ds, test_ds = spec2ds(DataSpec(d=20, k=1, gtype=:ER,
                    noise=:Gaussian, mat=:COV))

In [ ]:
x, y = next_batch!(test_ds) |> gpu

@show size(x)
@show size(y)

In [ ]:
model(x)

# model params

In [ ]:
include("main.jl")

In [ ]:
import Printf

In [ ]:
function test_size()
    # model size
    @info "FC model"
    for d in [10,20,50,100]
        Printf.@printf "%.2f\n" param_count(fc_model_fn(d)) / 1e6
    end
    @info "FC deep model"
    for d in [7, 10,15,20,25,30]
        Printf.@printf "%.2f\n" param_count(deep_fc_model_fn(d)) / 1e6
    end
    # EQ models is independent of input size
    @info "EQ model"
    Printf.@printf "%.2f\n" param_count(eq_model_fn(10)) / 1e6
    Printf.@printf "%.2f\n" param_count(deep_eq_model_fn(10)) / 1e6
end

In [ ]:
            ("EQ2", eq2_model_fn, 3e4),
            ("FC", ()->fc_model(d=d, ch=2, z=1024, nlayer=6), 3e4),
            ("FCreg", ()->fc_model(d=d, ch=2, z=1024, nlayer=6, reg=true), 3e4),
            ("CNN", ()->cnn_model(2), 3e4)

In [ ]:
for d in [10,20,50,100]
    Printf.@printf "%.2f\n" param_count(fc_model(d=d, ch=2, z=1024, nlayer=6)) / 1e6
end

In [ ]:
Printf.@printf "%.2f\n" param_count(eq2_model_fn()) / 1e6

In [ ]:
Printf.@printf "%.2f\n" param_count(eq_model_fn()) / 1e6

In [ ]:
Printf.@printf "%.2f\n" param_count(cnn_model(2)) / 1e6

In [ ]:
Printf.@printf "%.2f\n" param_count(cnn_model(1, 32)) / 1e6

In [ ]:
Printf.@printf "%.2f\n" param_count(cnn_model(2, 32)) / 1e6

In [ ]:
Printf.@printf "%.2f\n" param_count(cnn_model(2, 64)) / 1e6

In [ ]:
Printf.@printf "%.2f\n" param_count(cnn_model(2, 128)) / 1e6

In [ ]:
Printf.@printf "%.2f\n" param_count(cnn_model(2, 32, (5,5), (2,2))) / 1e6

In [ ]:
Printf.@printf "%.2f\n" param_count(cnn_model(2, 64, (5,5), (2,2))) / 1e6

In [ ]:
Printf.@printf "%.2f\n" param_count(cnn_model(2, 128, (5,5), (2,2))) / 1e6

# Test ERSF124 on individual types

In [ ]:
function test_ersf124()
    for d in [10, 20],
        types in [(:ER, :SF), (:ER2, :SF2), (:ER4, :SF4)]
        
        ID = "EQ2-ERSF124-d=$d-ensemble"

        specs = []

        for gtype in types,
            k in [1]
            push!(specs, DataSpec(d=d, k=k, gtype=gtype,
                                  noise=:Gaussian, mat=:CH3))
        end
        specs = Array{DataSpec}(specs)

        exp_test(ID, specs, "TEST-types=$types")
    end
end

In [ ]:
test_ersf124()

# Test d=100 model on d=100,200,300,400

In [ ]:
include("../src/main.jl")

In [ ]:
exp_test

In [ ]:
function test_large_d()
    for (ID,bsize) in [("EQ2-ERSF-k1-d=100-ensemble", 4),
                       ("EQ2-ERSF-k1-d=200-ng=1000-N=1-ensemble", 4),
                       ("EQ2-ERSF-k1-d=400-ng=1000-N=1-ensemble", 4)],
        d in [200, 400]

        specs = []

        for gtype in [:ER, :SF],
            k in [1]
            push!(specs, DataSpec(d=d, k=k, gtype=gtype,
                                  noise=:Gaussian, mat=:CH3,
            bsize=bsize))
        end
        specs = Array{DataSpec}(specs)

        exp_test(ID, specs, "TEST-d=$d")
    end
end

In [ ]:
test_large_d()

# Test the ensemble model on unseen d

In [ ]:
# can I just run the test without saving? No because loading the main.jl is slow

In [ ]:
include("main.jl")

In [ ]:
function test_unseen_d()
    for ID in ["EQ2-CH3-d=[10,15,20]-ensemble","EQ2-CH3-d=[20,30,40]-ensemble"],
        d in [10, 15, 20, 30, 40, 50, 80, 100]

        specs = []

        for gtype in [:ER, :SF],
            k in [1]
            push!(specs, DataSpec(d=d, k=k, gtype=gtype,
                                  noise=:Gaussian, mat=:CH3))
        end
        specs = Array{DataSpec}(specs)

        exp_test(ID, specs, "TEST-d=$d")
    end
end

In [ ]:
test_unseen_d()

In [ ]:
# test the individual trained models
# d=10,15,20
# on larger d
function test_unseen_d_2()
    for ID in ["EQ2-ERSF-k1-d=10-ensemble", "EQ2-ERSF-k1-d=15-ensemble", "EQ2-ERSF-k1-d=20-ensemble"],
        d in [10, 15, 20, 30, 40, 50, 80, 100]

        specs = []

        for gtype in [:ER, :SF],
            k in [1]
            push!(specs, DataSpec(d=d, k=k, gtype=gtype,
                                  noise=:Gaussian, mat=:CH3))
        end
        specs = Array{DataSpec}(specs)

        exp_test(ID, specs, "TEST-d=$d")
    end
end

In [ ]:
test_unseen_d_2()

# Test on different noise and the tables

In [ ]:
function test_different_noise()
    # FIXME OOM for d=100??
    for d in [10,20,50,100],
        noise in [:Gaussian, :Poisson, :Exp, :Gumbel]

        ID = "EQ2-ERSF-k1-d=$d-ensemble"

        specs = []
        for gtype in [:ER, :SF],
            k in [1]
            push!(specs, DataSpec(d=d, k=k, gtype=gtype,
                    noise=noise, mat=:CH3))
        end
        specs = Array{DataSpec}(specs)
        
        @info "Testing on" ID d noise
        exp_test(ID, specs, "TEST-d=$d-noise=$noise")
    end
end

In [ ]:
test_different_noise()

# Sachs 2005 experiment

## Load and explore the data

In [ ]:
import CSV

In [ ]:
df = CSV.read("Sachs/1.cd3cd28.csv")

In [ ]:
SachsX = convert(Matrix, df)

In [ ]:
include("data_graph.jl")

In [ ]:
SachsG = Sachs_ground_truth()

In [ ]:
myplot(SachsG)

In [ ]:
medcovX = cov(SachsX) ./ median(var(SachsX, dims=1))

In [ ]:
maxcovX = cov(SachsX) ./ maximum(var(SachsX, dims=1))

In [ ]:
corX = cor(SachsX)

In [ ]:
ch2X = getch2(SachsX)

In [ ]:
include("exp.jl")

## Load a model

In [ ]:
# load the trained model
@load "saved_models/EQ-d=20_k=1_gtype=SF_noise=Gaussian_mat=medCOV_mec=Linear/step-15000.bson" model

In [ ]:
@load "saved_models/EQ-d=10_k=1_gtype=ER_noise=Gaussian_mat=medCOV_mec=Linear/step-15000.bson" model

In [ ]:
@load "back/back-0907/CORCOV/EQ-d=10_k=1_gtype=SF_noise=Gaussian_mat=COR_mec=Linear/step-15000.bson" model

In [ ]:
@load "saved_models/ensK-2020-09-08T10:58:41.247-ensemble/step-10000.bson" model

In [ ]:
# The new ensemble model
@load "saved_models/ensemEQ-ICLR-1-ensemble/step-159443.bson" model

In [ ]:
@load "saved_models/ensemEQ-CH2-1,2,4-2020-10-11T11:29:01.183-ensemble/step-100000.bson" model

## Evaluate the model

In [ ]:
out = inf_one(model, medcovX)

In [ ]:
out = inf_one(model, corX)

In [ ]:
out = inf_one(model, maxcovX)

In [ ]:
out = inf_one(model, ch2X)

In [ ]:
Wout = threshold(σ.(out), 0.3, true)

In [ ]:
myplot(DiGraph(Wout), names(df))

In [ ]:
# FIXME NOW !!!! the names might not match at all!!!
myplot(SachsG, names(df))

In [ ]:
# predicted edge, true edge, SHD
predicted_edge = ne(DiGraph(Wout))
@show predicted_edge
correct_edge = sum(Wout[Wout .== 1] .== adjacency_matrix(SachsG)[Wout .== 1])
@show correct_edge

# metrics
ytrue = Matrix(gen_weights(SachsG))
sup_graph_metrics(Wout, ytrue)

In [ ]:
# TODO calculate #reverse direction edges
sum(Wout[Wout .== 1] .== adjacency_matrix(SachsG)[Wout .== 1])

In [ ]:
sum(Wout'[Wout' .== 1] .== adjacency_matrix(SachsG)[Wout' .== 1])

In [ ]:
# TODO implement the recursive add procedure to remove cycles
# Or, just construct the graph, and keep removing until it is a DAG
is_cyclic(DiGraph(Wout))

In [ ]:
Wout

In [ ]:
# Or just implement the procedure
out

## Adding non-cyclic procedure

In [ ]:
Wout2 = threshold(σ.(out), 0.3, false)

In [ ]:
# find the order of the index, or, sort the indexes
sort(Wout2, dims=1)

In [ ]:
edgeidx = findall((x)->x>0, Wout2)

In [ ]:
sorted_idx = edgeidx[sortperm(Wout2[edgeidx], rev=true)]

In [ ]:
Wout2

In [ ]:
# add sorted idx
g = MetaDiGraph(11)
for idx in sorted_idx
    add_edge!(g, idx[1], idx[2])
    if is_cyclic(g)
        rem_edge!(g, idx[1], idx[2])
    end
end

In [ ]:
p1 = myplot(g, names(df))

In [ ]:
draw(PNG("p1.png"), p1)

In [ ]:
p0 = myplot(SachsG, names(df))

In [ ]:
draw(PNG("p0.png"), p0)

In [ ]:
adj1 = adjacency_matrix(g)
adj0 = adjacency_matrix(SachsG)
# predicted edge, true edge, SHD
predicted_edge = ne(g)
@show predicted_edge
correct_edge = sum(adj1[adj1 .== 1] .== adj0[adj1 .== 1])
@show correct_edge
reversed_edge = sum(adj1'[adj1' .== 1] .== adj0[adj1' .== 1])
@show reversed_edge

# metrics
ytrue = Matrix(gen_weights(SachsG))
sup_graph_metrics(adj1, ytrue)

## More data

- [ ] goblinop's gaussian data (just for test)
- [ ] bnlearn's continous data
- [ ] discrete data?

### goblinop's (??) gaussian data

In [ ]:
import CSV

In [ ]:
df = CSV.read("data-back/gaussian.dat", delim=" ")

In [ ]:
X = convert(Matrix, df)

In [ ]:
include("data_graph.jl")

In [ ]:
maxcovX = cov(X) ./ maximum(var(X, dims=1))

In [ ]:
include("exp.jl")

In [ ]:
# The new ensemble model
@load "saved_models/ensemEQ-ICLR-1-ensemble/step-159443.bson" model

In [ ]:
out = inf_one(model, maxcovX)

In [ ]:
Wout2 = threshold(σ.(out), 0.3, false)

In [ ]:
# find the order of the index, or, sort the indexes
sort(Wout2, dims=1)

In [ ]:
edgeidx = findall((x)->x>0, Wout2)

In [ ]:
sorted_idx = edgeidx[sortperm(Wout2[edgeidx], rev=true)]

In [ ]:
Wout2

In [ ]:
size(X,2)

In [ ]:
# add sorted idx
g = MetaDiGraph(size(X,2))
for idx in sorted_idx
    add_edge!(g, idx[1], idx[2])
    if is_cyclic(g)
        rem_edge!(g, idx[1], idx[2])
    end
end

In [ ]:
bnp1 = myplot(g, ["A", "B", "C", "D", "E", "F", "G"])

In [ ]:
function bnlearn_ground_truth()
    greal = named_graph([:A, :B, :C, :D, :E, :F, :G])
    named_graph_add_edge!(greal, :B, :C)
    named_graph_add_edge!(greal, :A, :C)
    named_graph_add_edge!(greal, :B, :D)
    named_graph_add_edge!(greal, :D, :F)
    named_graph_add_edge!(greal, :A, :F)
    named_graph_add_edge!(greal, :G, :F)
    named_graph_add_edge!(greal, :E, :F)
    greal
end

In [ ]:
groundG = bnlearn_ground_truth()

In [ ]:
bnp0 = myplot(groundG, ["A", "B", "C", "D", "E", "F", "G"])

In [ ]:
draw(PNG("bnp0.png"), bnp0)

In [ ]:
draw(PNG("bnp1.png"), bnp1)

In [ ]:
adj1 = adjacency_matrix(g)
adj0 = adjacency_matrix(groundG)
# predicted edge, true edge, SHD
predicted_edge = ne(g)
@show predicted_edge
correct_edge = sum(adj1[adj1 .== 1] .== adj0[adj1 .== 1])
@show correct_edge
reversed_edge = sum(adj1'[adj1' .== 1] .== adj0[adj1' .== 1])
@show reversed_edge

# metrics
ytrue = Matrix(gen_weights(groundG))
sup_graph_metrics(adj1, ytrue)